<a href="https://colab.research.google.com/github/achett/Hierarchical-Model/blob/main/Bayesian_Hierarchical_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hierarchicalforecast
!pip install statsforecast
!pip install datasetsforecast
!pip install nixtlats>=0.1.0
!pip install darts
!pip install mlforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.2/508.2 kB 35.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.85 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.0 MB/s eta 0:00:00
  Created wheel for fugue-sql-antlr: filename=fugue_sql_antlr-0.2.0-py3-none-any.whl size=158196 sha256=fc43bd0e4d16d06ff9dce45af76e2840e40d826d29b1ae08d18858c77b287938
  Stored in directory: /root/.cache/pip/wheels/5a/b5/4e/216953a1c711da55de29ed7ecf158b4a5bf32ef93d69ad66dd
Successfully built fugue-sql

In [3]:
########################
# PACKAGES
########################
# !pip install -U numba statsforecast datasetsforecast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
from datetime import datetime
import pymc as pm

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive, AutoETS, AutoCES, AutoTheta

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut, MinTrace, OptimalCombination, ERM, PERMBU
from hierarchicalforecast.utils import aggregate

# #obtain hierarchical dataset
# from datasetsforecast.hierarchical import HierarchicalData
from nixtlats import TimeGPT
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'

import pandas as pd
from darts import TimeSeries
from darts.models import RegressionModel
from darts.dataprocessing.transformers import Scaler
from lightgbm import LGBMRegressor
from darts.metrics import mae, rmse
from darts.models import LightGBMModel
from darts import concatenate
# Import necessary libraries
import pandas as pd
from darts import TimeSeries
from darts.models import ExponentialSmoothing, StatsForecastAutoETS, StatsForecastAutoARIMA, KalmanForecaster
from darts.metrics import mae, mape

import pandas as pd
from darts import TimeSeries
from darts.models import RegressionModel
from darts.dataprocessing.transformers import Scaler
from lightgbm import LGBMRegressor
from darts.models import LightGBMModel
from darts import concatenate
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from darts.utils.likelihood_models import QuantileRegression


pd.options.display.float_format = '{:,.2f}'.format

In [4]:
##############
# PARAMS
##############
fct_periods = 12
fct_st_date = '2023-04-01'
fct_end_date = '2023-12-01'

# Create hierarchical structure and constraints
hierarchy_levels = [['top_level'],
                    ['top_level', 'level2'],
                    ['top_level', 'level2', 'level3'],
                    ['top_level', 'level2', 'level3', 'bottom_level']]

inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/regional_hierarchy.csv'
r_hier = pd.read_csv(inputFile)


In [5]:
##############
# FUNCTIONS
##############

def prepare_data(data, r_hier):
    # Merge hierarchy
    data = data.merge(r_hier, how='left', left_on='cost_object', right_on='cost_object')

    # Transform date and y
    data['ds'] = pd.to_datetime(data['ds'])
    data['y'] = data['y'].astype(float)

    # Address NA values
    data['volume'] = data['y'].fillna(0)
    data['region'] = data['region'].fillna('')
    data['cost_object'] = data['cost_object'].fillna('')
    data['product'] = data['product'].fillna('')

    # Create hierarchical dataframe
    data['top_level'] = 'global'  # Assuming 'top_level' does not contain '/', otherwise add a replace line for it too.
    data.rename(columns={'region': 'level2', 'cost_object': 'level3', 'product': 'bottom_level'}, inplace=True)
    data = data[['level2', 'level3', 'bottom_level', 'top_level', 'ds', 'y']]

    # Replace '/' with '_' in the four columns
    data['level2'] = data['level2'].str.replace('/', '_')
    data['level3'] = data['level3'].str.replace('/', '_')
    data['bottom_level'] = data['bottom_level'].str.replace('/', '_')

    data['unique_id'] = data['top_level'] + '/' + data['level2'] + '/' + data['level3'] + '/' + data['bottom_level']

    return data

def prepare_feature(data, r_hier, volume_act2, feature_name):

    # Select and rename columns
    data = data[['cost_object', 'product', 'ds', feature_name]].rename(columns={feature_name: 'y'})

    # Apply any additional preparation (assuming prepare_data is a function you have defined)
    data = prepare_data(data, r_hier)

    # Rename the columns back
    data = data.rename(columns={'y': feature_name})

    # Merge with the volume_act2 dataframe
    merged_df = data.merge(volume_act2[['unique_id', 'ds']], how='right', on=['unique_id', 'ds'])

    return merged_df


In [6]:
##############
# DATA LOAD
##############
inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/budgetFY23.csv'
budget = pd.read_csv(inputFile)
# budget = budget[budget['category']=='EQUIV_UNIT - Equivalent Units']
budget = budget[budget['category']=='UC110000 - Total Revenue']
budget.rename(columns={'country': 'cost_object'}, inplace=True)
budget = prepare_data(budget, r_hier)

inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/revenue_output.csv'
volume_act = pd.read_csv(inputFile)
volume_act.rename(columns={'value': 'y'}, inplace=True)
volume_act = prepare_data(volume_act, r_hier)

inputFile = '/content/drive/MyDrive/Colab Notebooks/SGA Prediction/data/sga_output.csv'
sga = pd.read_csv(inputFile)

sga1 = prepare_feature(sga, r_hier, volume_act, 'AP')
sga2 = prepare_feature(sga, r_hier, volume_act, 'Field_Sales')


inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/ids2smooth.csv'
ids2smooth = pd.read_csv(inputFile)

<ipython-input-6-0a79675a25c2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget.rename(columns={'country': 'cost_object'}, inplace=True)


In [7]:
########################
# SAMPLE
########################
# Subset
regs2include = ['global/North America/US10 - Astellas Pharma US, Inc./ENZA - Enzalutamide', 'global/North America/US10 - Astellas Pharma US, Inc./REGADENOSN - Regadenoson']
volume_act = volume_act[volume_act['unique_id'].isin(regs2include)]

# level2include = ['CIS']
# volume_act = volume_act[volume_act['level2'].isin(level2include)]

In [8]:
########################
# IDENTIFY UNIVERSE
########################
tested_ts = set(budget['unique_id'].unique()).intersection(volume_act['unique_id'].unique())

# Find unique IDs present in budget_h but not in rev
unique_ids_in_budget_not_in_rev = set(budget['unique_id'].unique()).difference(volume_act['unique_id'].unique())

# Find unique IDs present in rev but not in budget_h
unique_ids_in_rev_not_in_budget = set(volume_act['unique_id'].unique()).difference(budget['unique_id'].unique())

# Filter volume
volume_act = volume_act[volume_act['unique_id'].isin(tested_ts)]

In [9]:
########################
# SEGMENT TIME SERIES
########################
new_products = ['ENFORTUMAB - Enforumab Vedotin', 'ROXADUSTNT - Roxadustant']
loe_products = ['REGADENOSN - Regadenoson']
div_products = ['MICAFUNGIN - Micafungin Sodium']

new_ids = volume_act[volume_act['bottom_level'].isin(new_products)]['unique_id'].unique().tolist()
loe_ids = volume_act[volume_act['bottom_level'].isin(loe_products)]['unique_id'].unique().tolist()
divested_ids = volume_act[volume_act['bottom_level'].isin(div_products)]['unique_id'].unique().tolist()

# IDs with A&P and Field Sales Spend
grouped1 = sga1.groupby('unique_id')[['AP']].sum()
grouped2 = sga2.groupby('unique_id')[['Field_Sales']].sum()
spend_ids = set(grouped1[(grouped1['AP'] > 0)].index.tolist() + grouped2[(grouped2['Field_Sales'] > 0)].index.tolist())
spend_ids = spend_ids.difference(new_ids + loe_ids + divested_ids)

# IDs with no spend
non_spend_ids = volume_act[~volume_act['unique_id'].isin(spend_ids)]['unique_id'].unique()

arima_regions = ['Japan', 'BENELUX', 'North America', 'Nordic', 'Baltic']
ets_regions = ['East Europe', 'West Europe', 'Greater China', 'MEA', 'LATAM', 'SESA', 'CIS']
# arima_ids = volume_act[(volume_act['level2'].isin(arima_regions)) & (~volume_act['unique_id'].isin(spend_ids))]['unique_id'].unique().tolist()
# ets_ids = volume_act[(volume_act['level2'].isin(ets_regions)) & (~volume_act['unique_id'].isin(spend_ids))]['unique_id'].unique().tolist()

arima_ids = volume_act[(volume_act['level2'].isin(arima_regions))]['unique_id'].unique().tolist()
ets_ids = volume_act[(volume_act['level2'].isin(ets_regions))]['unique_id'].unique().tolist()

In [10]:
########################
# INTERMITTENT DEMAND CANDIDATES
########################

# Function to calculate the percentage of zeros after the first non-zero
def calculate_percentage_zeros(df):
    # Find the index of the first non-zero entry in 'y'
    first_non_zero_index = df.loc[df['y'] != 0].index.min()
    # If there are no non-zero values, return None or 0 based on your preference
    if pd.isna(first_non_zero_index):
        return None  # Or return 0 if you want to treat this as 0% zeros following non-zero
    # Select the subset of 'y' after the first non-zero
    post_non_zero_series = df.loc[first_non_zero_index:, 'y']
    # Count the number of zeros in this subset
    num_zeros = (post_non_zero_series == 0).sum()
    # Calculate the percentage of zeros
    percentage_zeros = num_zeros / len(post_non_zero_series) * 100
    return percentage_zeros

# Apply the function to each group and reset index to make unique_id a column
percentage_zeros_df = volume_act.groupby('unique_id').apply(calculate_percentage_zeros).reset_index(name='percentage_zeros')

inter_demand_ids = percentage_zeros_df[percentage_zeros_df['percentage_zeros']>=50]['unique_id'].tolist()


In [35]:
########################
# RUN ETS & ARIMA
########################
def convert_fct2df(forecasts):
    forecast_dfs = []
    for unique_id, forecast_ts in forecasts.items():
        df = TimeSeries.quantiles_df(forecast_ts, quantiles=[0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.975, 0.995])
        df['unique_id'] = unique_id
        df = df.reset_index()
        df = df.rename(columns={'y_0.5': 'y'})
        forecast_dfs.append(df)

    # Concatenate all forecast DataFrames into a single DataFrame
    all_forecasts_df = pd.concat(forecast_dfs, axis=0)

    # Reorder and rename columns as needed
    columns = ['unique_id'] + [col for col in all_forecasts_df.columns if col != 'unique_id']
    all_forecasts_df = all_forecasts_df[columns]

    all_forecasts_df.columns.name = None

    return all_forecasts_df

def generate_time_series_dict(data, fct_periods, filter_data):
    # Split train/test sets
    test = data.groupby('unique_id').tail(fct_periods)
    train = data.drop(test.index)

    # Prepare time series dataframes
    time_series_dfs = {uid: group for uid, group in train.groupby('unique_id')}
    time_series_dict = {}

    if filter_data:
        # Filter out time series with insufficient non-zero data points
        filtered_time_series_dfs = {}
        for uid, group in time_series_dfs.items():
            non_zero_index = group['y'].ne(0).idxmax()
            start_index = max(0, non_zero_index - (13 - 1))
            filtered_df = group.loc[non_zero_index:] if group.loc[non_zero_index:].shape[0] >= 13 else group.loc[start_index:]
            if not filtered_df.empty:
                filtered_time_series_dfs[uid] = filtered_df
        # Convert each filtered DataFrame into a Darts TimeSeries object
        time_series_dict = {uid: TimeSeries.from_dataframe(group, 'ds', 'y') for uid, group in filtered_time_series_dfs.items()}
    else:
        # Convert each original DataFrame into a Darts TimeSeries object without filtering
        time_series_dict = {uid: TimeSeries.from_dataframe(group, 'ds', 'y') for uid, group in time_series_dfs.items()}

    return time_series_dict

def generate_forecast(data, fct_periods, model2use, filter_data=True):

    # Use the nested function to generate the time series dictionary
    time_series_dict = generate_time_series_dict(data, fct_periods, filter_data)

    # Create and fit a model for each time series
    models = {}
    for uid, series in time_series_dict.items():
        model = get_model(model2use)
        model.fit(series)
        models[uid] = model

    # Forecasting
    fct_dict = {uid: model.predict(fct_periods, num_samples=20) for uid, model in models.items()}
    print(model.predict(fct_periods, num_samples=20))
    # Convert forecasts into a dataframe
    fct_df = convert_fct2df(fct_dict)

    return fct_dict, fct_df

# Function to dynamically get the model instance
def get_model(model_name):
    if model_name == 'AutoETS':
        return StatsForecastAutoETS()
    elif model_name == 'ARIMA':
        return StatsForecastAutoARIMA(season_length=12)
    elif model_name == 'KF':
        return KalmanForecaster(dim_x=12)
    else:
        raise ValueError(f"Unsupported model: {model_name}")

ets_dict, ets_df = generate_forecast(volume_act, fct_periods, model2use='AutoETS', filter_data=True)
arima_dict, arima_df = generate_forecast(volume_act, fct_periods, model2use='ARIMA', filter_data=True)
# kf_fct = generate_forecast(volume_act, fct_periods, model2use='KF', filter_data=True)


<TimeSeries (DataArray) (ds: 12, component: 1, sample: 20)>
array([[[7.15509304e+09, 8.55236206e+09, 6.68102193e+09, 6.15715924e+09,
         5.58093371e+09, 5.14728576e+09, 6.40739119e+09, 6.10017044e+09,
         7.79088183e+09, 5.20085695e+09, 8.79086835e+09, 6.81455415e+09,
         6.54999213e+09, 3.79395830e+09, 5.51481927e+09, 6.44236941e+09,
         5.23396585e+09, 6.20653279e+09, 8.56393770e+09, 5.48400139e+09]],

       [[6.37124954e+09, 6.37776940e+09, 6.83728007e+09, 6.58903613e+09,
         6.52388193e+09, 7.62552487e+09, 6.62085444e+09, 7.00365240e+09,
         5.66909339e+09, 6.67201016e+09, 6.49846849e+09, 7.00971178e+09,
         6.47637710e+09, 6.81276453e+09, 6.54582858e+09, 5.89978522e+09,
         8.83347813e+09, 5.90306734e+09, 6.03816226e+09, 7.19640868e+09]],

       [[6.45666435e+09, 8.91651275e+09, 6.00810887e+09, 8.11402876e+09,
         6.25778419e+09, 5.70336438e+09, 6.53893700e+09, 8.91469940e+09,
         6.32756779e+09, 5.36598136e+09, 5.27254872e+09, 6

In [12]:
from darts.metrics import quantile_loss, mse, ope
from darts import TimeSeries
import pandas as pd

def sum_of_differences(time_series):
    # Find the index of the first non-zero value
    first_non_zero_index = next((index for index, value in enumerate(time_series) if value != 0), None)

    # Check if there is a non-zero value in the series
    if first_non_zero_index is None:
        return 0  # Return 0 if there are no non-zero values

    # Calculate the sum of the differences after the first non-zero value
    sum_diff = sum(abs(time_series[i] - time_series[i - 1]) for i in range(first_non_zero_index + 1, len(time_series)))

    # Calculate the number of time points after the first non-zero value minus one
    num_points = len(time_series) - first_non_zero_index - 1

    # Avoid division by zero
    if num_points <= 0:
        return 0

    # Return the result
    return np.array((sum_diff / num_points).values())[0][0]


def metrics(actual_data, forecasted_data, quantiles, weights):
    # Prepare a list to store WSPL results for each unique_id
    results = []

    # Ensure 'ds' is in datetime format
    actual_data['ds'] = pd.to_datetime(actual_data['ds'])

    for unique_id in actual_data['unique_id'].unique():
        # Filter the actual data and forecasted data
        actual_values = actual_data[actual_data['unique_id'] == unique_id][['ds', 'y']].tail(fct_periods)
        actual_ts = TimeSeries.from_dataframe(actual_values.set_index('ds'))

        historical_actuals = actual_data[actual_data['unique_id'] == unique_id][['ds', 'y']].drop(actual_values.index)

        historical_ts = TimeSeries.from_dataframe(historical_actuals.set_index('ds'))
        forecasted_ts = forecasted_data[unique_id]

        # Calculate quantile loss for each quantile
        losses = {q: quantile_loss(actual_ts, forecasted_ts, q) for q in quantiles}
        print(forecasted_ts)
        wspl = sum(weights[q] * losses[q] for q in quantiles) / sum(weights.values())

        # Calculate rmse
        mse_metric = mse(actual_ts, forecasted_ts)
        scaled = sum_of_differences(historical_ts)

        rmse_metric = np.sqrt(mse_metric)
        rmsse_metric = np.sqrt((mse_metric/scaled))

        # Calculate the overall percentage error
        try:
            ope_metric = ope(actual_ts, forecasted_ts)
        except ValueError:
            ope_metric = np.nan


        # Append the result to the list
        results.append({'unique_id': unique_id, 'WSPL': wspl, 'RMSE': rmse_metric, 'RMSSE': rmsse_metric, 'OPE': ope_metric})


    # Convert the list of results into a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Usage:
quantiles = [0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.975, 0.995] # Define your quantiles
weights = {0.005: 1, 0.025: 1, 0.165: 1, 0.250: 1, 0.500: 1, 0.750: 1, 0.835: 1, 0.975: 1, 0.995: 1}

results = metrics(volume_act, arima_dict, quantiles, weights)
results


<TimeSeries (DataArray) (ds: 12, component: 1, sample: 20)>
array([[[2.31386818e+10, 2.69236851e+10, 2.47148425e+10, 2.88233532e+10,
         2.37316941e+10, 2.55866686e+10, 2.82675683e+10, 2.30943955e+10,
         2.99259496e+10, 2.53864178e+10, 2.78435571e+10, 2.42285137e+10,
         2.09457546e+10, 2.28811040e+10, 2.82681083e+10, 2.48148930e+10,
         2.41020411e+10, 1.94213938e+10, 2.45405307e+10, 2.30860839e+10]],

       [[2.21089434e+10, 2.53355160e+10, 2.38130522e+10, 2.09984766e+10,
         2.11605702e+10, 2.28855850e+10, 2.11433736e+10, 2.00576743e+10,
         2.22673217e+10, 2.05280723e+10, 2.57228142e+10, 2.70892986e+10,
         2.33470210e+10, 2.59356442e+10, 2.32719895e+10, 2.58110734e+10,
         2.25709889e+10, 2.26539173e+10, 2.08745106e+10, 2.28168223e+10]],

       [[2.47320170e+10, 2.34513974e+10, 2.12187137e+10, 2.42787500e+10,
         2.58541253e+10, 2.26512401e+10, 2.99338337e+10, 2.88484695e+10,
         2.58904079e+10, 2.04043832e+10, 2.46760967e+10, 2

,unique_id,WSPL,RMSE,RMSSE,OPE
0,global/North America/US10 - Astellas Pharma US...,"1,843,638,296.84","3,595,679,072.66","70,157.84",1.30
1,global/North America/US10 - Astellas Pharma US...,"2,300,063,978.11","4,991,053,887.02","164,037.93",181.24


In [34]:
arima_dict['global/North America/US10 - Astellas Pharma US, Inc./REGADENOSN - Regadenoson']

<TimeSeries (DataArray) (ds: 12, component: 1, sample: 20)>
array([[[6.75554884e+09, 5.02639700e+09, 6.71533790e+09, 5.83282971e+09,
         4.24041269e+09, 4.37917132e+09, 5.76634896e+09, 4.30804752e+09,
         6.13875820e+09, 3.70003616e+09, 6.78201397e+09, 4.85236381e+09,
         6.67746515e+09, 3.64935154e+09, 5.37203173e+09, 5.47417654e+09,
         7.01804051e+09, 5.68687461e+09, 5.50143708e+09, 6.11847196e+09]],

       [[5.51850505e+09, 6.35671542e+09, 6.74356872e+09, 5.68833080e+09,
         4.70816240e+09, 7.26216508e+09, 6.32605432e+09, 6.17422088e+09,
         7.34919166e+09, 6.76112846e+09, 6.04118112e+09, 6.42513665e+09,
         6.77313012e+09, 6.40588103e+09, 5.72286039e+09, 7.73376463e+09,
         8.13819908e+09, 6.34999000e+09, 6.44085979e+09, 5.98490041e+09]],

       [[6.96133249e+09, 7.04569097e+09, 8.10066706e+09, 5.89201791e+09,
         7.83502175e+09, 8.41174252e+09, 6.07553812e+09, 8.57659069e+09,
         8.37525294e+09, 7.47581309e+09, 5.60586856e+09, 6.36864942e+09,
         7.39676066e+09, 7.29054748e+09, 6.10642159e+09, 7.42617855e+09,
         7.91880047e+09, 5.34325109e+09, 8.86801990e+09, 9.10382008e+09]],

       [[5.32392425e+09, 7.92855452e+09, 7.25528785e+09, 7.77442008e+09,
         5.30436765e+09, 7.54064539e+09, 6.81417612e+09, 7.02237021e+09,
...
         7.95610150e+09, 6.66684217e+09, 6.30774055e+09, 6.06185384e+09,
         7.68411018e+09, 8.56367089e+09, 5.95362281e+09, 6.61091282e+09]],

       [[5.21305408e+09, 5.22937183e+09, 6.48236167e+09, 6.35761407e+09,
         6.15420756e+09, 7.43602697e+09, 8.26634993e+09, 5.69651693e+09,
         6.20402204e+09, 6.36936872e+09, 8.14953391e+09, 7.10724281e+09,
         5.92212444e+09, 5.89545498e+09, 6.02184167e+09, 7.42151086e+09,
         7.36372457e+09, 4.71195899e+09, 6.73740888e+09, 8.10315697e+09]],

       [[6.69073069e+09, 8.05174822e+09, 5.37377071e+09, 6.03844840e+09,
         5.54787063e+09, 6.42440808e+09, 6.27148490e+09, 4.50390761e+09,
         6.80965522e+09, 6.04036249e+09, 6.29537805e+09, 8.44296518e+09,
         8.19361130e+09, 5.28641305e+09, 7.08427299e+09, 6.89392804e+09,
         6.83828606e+09, 5.42958765e+09, 5.08588430e+09, 5.80252019e+09]],

       [[7.22276088e+09, 5.27312747e+09, 5.03037952e+09, 4.69857002e+09,
         4.90149214e+09, 8.48780082e+09, 6.66361151e+09, 6.40482365e+09,
         6.98022824e+09, 4.88012547e+09, 6.35916158e+09, 4.57715324e+09,
         3.84426056e+09, 6.48299913e+09, 5.29351168e+09, 4.91189886e+09,
         5.66908305e+09, 7.57639569e+09, 6.74345630e+09, 7.96143226e+09]]])
Coordinates:
  * ds         (ds) datetime64[ns] 2023-01-01 2023-02-01 ... 2023-12-01
  * component  (component) object 'y'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [13]:
########################
# RUN ENSEMBLE
########################
# from darts.models import RegressionEnsembleModel, NaiveSeasonal, LinearRegressionModel
# import pandas as pd
# from darts import TimeSeries
# from darts.models import ExponentialSmoothing, StatsForecastAutoETS, StatsForecastAutoTheta, StatsForecastAutoARIMA
# from darts.metrics import mae, mape

# train2 = train[train['unique_id'].isin(['global/Japan/JP10 - Astellas Pharma Inc/ENZA - Enzalutamide', 'global/North America/US10 - Astellas Pharma US, Inc./ENZA - Enzalutamide', 'global/West Europe/D_E_FRANCE - France/ROXADUSTNT - Roxadustant'])]

# time_series_dfs = {uid: group for uid, group in train.groupby('unique_id')}

# # Convert each filtered DataFrame into a Darts TimeSeries object
# time_series_dict = {uid: TimeSeries.from_dataframe(group, 'ds', 'y') for uid, group in time_series_dfs.items()}

# # Create and fit model for each time series
# ensemble_models = {}

# for uid, series in time_series_dict.items():
#     model = RegressionEnsembleModel(forecasting_models = [StatsForecastAutoETS(season_length=12), StatsForecastAutoARIMA(season_length=12)],regression_train_n_points=36)

#     model.fit(series)
#     ensemble_models[uid] = model

# # Forecasting (assuming you want to forecast the next N steps)
# N = 12
# forecasts = {uid: model.predict(N) for uid, model in ensemble_models.items()}

# ensemble_fct = convert_fct2df(forecasts)

In [14]:
########################
# RUN QUARTERLY MODEL
########################
# Function to resample and sum data by quarter for each group
def resample_group(group):
    group = group.set_index('ds')  # Set 'ds' as the index
    resampled_group = group.resample('Q').agg({'y': 'sum'})  # Aggregate data by quarter
    return resampled_group

quarterly_data = volume_act
quarterly_data['ds'] = pd.to_datetime(quarterly_data['ds'])

# Group by 'unique_id' and apply the resampling function
quarterly_data = quarterly_data.groupby('unique_id').apply(resample_group).reset_index()



In [15]:
########################
# FEATURE ENGINEERING
########################
# Feature Engineering
inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/volume_output.csv'
features = pd.read_csv(inputFile)

inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/price_output.csv'
price = pd.read_csv(inputFile)

inputFile = '/content/drive/MyDrive/Colab Notebooks/SGA Prediction/data/sga_output.csv'
sga = pd.read_csv(inputFile)

sga1 = prepare_feature(sga, r_hier, volume_act, 'AP')
sga2 = prepare_feature(sga, r_hier, volume_act, 'Field_Sales')
sga3 = prepare_feature(sga, r_hier, volume_act, 'Smoothed_AP')
sga4 = prepare_feature(sga, r_hier, volume_act, 'Smoothed_Field_Sales')

price1 = prepare_feature(price, r_hier, volume_act, 'Price')
price2 = prepare_feature(price, r_hier, volume_act, 'Smoothed_Price')


features = features[['cost_object', 'product', 'ds', 'value']]
features.rename(columns={'value': 'y'}, inplace=True)
features = prepare_data(features, r_hier)
features = features.merge(volume_act[['unique_id', 'ds']], how='right', on=['unique_id', 'ds'])

# Encoding 'cost_object' and 'product'
label_encoder = LabelEncoder()
features['region'] = label_encoder.fit_transform(features['level2'])
features['cost_object'] = label_encoder.fit_transform(features['level3'])
features['product'] = label_encoder.fit_transform(features['bottom_level'])
features = features[['unique_id', 'ds', 'region', 'cost_object', 'product', 'top_level']]

<ipython-input-15-f19d57e6163a>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.rename(columns={'value': 'y'}, inplace=True)


In [16]:
########################
# RUN LGBM
########################
# time_series_dict = generate_time_series_dict(volume_act, fct_periods, filter_data=False)

# # Prepare individual time series for dynamic features
# dynamic_features_dict = {}
# for uid in sga1['unique_id'].unique():
#     df1 = sga1[sga1['unique_id'] == uid].sort_values('ds')
#     df2 = sga2[sga2['unique_id'] == uid].sort_values('ds')
#     df3 = sga3[sga3['unique_id'] == uid].sort_values('ds')
#     df4 = sga4[sga4['unique_id'] == uid].sort_values('ds')
#     df5 = price1[price1['unique_id'] == uid].sort_values('ds')
#     df6 = price2[price2['unique_id'] == uid].sort_values('ds')

#     # Create a TimeSeries object for features
#     ts1 = TimeSeries.from_dataframe(df1[['ds', 'AP']], 'ds', 'AP')
#     ts2 = TimeSeries.from_dataframe(df2[['ds', 'Field_Sales']], 'ds', 'Field_Sales')
#     ts3 = TimeSeries.from_dataframe(df3[['ds', 'Smoothed_AP']], 'ds', 'Smoothed_AP')
#     ts4 = TimeSeries.from_dataframe(df4[['ds', 'Smoothed_Field_Sales']], 'ds', 'Smoothed_Field_Sales')
#     ts5 = TimeSeries.from_dataframe(df5[['ds', 'Price']], 'ds', 'Price')
#     ts6 = TimeSeries.from_dataframe(df6[['ds', 'Smoothed_Price']], 'ds', 'Smoothed_Price')

#     # Concatenate the two TimeSeries along the feature axis (axis=1)
#     combined_ts = concatenate([ts1, ts2, ts3, ts6], axis=1)
#     # combined_ts = concatenate([ts3], axis=1)

#     # Store the combined TimeSeries in the dictionary
#     dynamic_features_dict[uid] = combined_ts



# # Create and train a regression model for each time series
# models = {}
# scalers = {}
# forecasts = {}
# for uid, ts_combined in time_series_dict.items():
#     # Scale the combined series
#     scaler = Scaler()
#     ts_combined_transformed = scaler.fit_transform(ts_combined)
#     scalers[uid] = scaler  # Store scaler for later inverse transformation

#     # Initialize and fit your LightGBM model
#     model = LightGBMModel(lags=len(ts_combined) // 2, output_chunk_length=1, verbose=-1, lags_past_covariates=12)
#     model.fit(series=ts_combined_transformed, past_covariates=dynamic_features_dict[uid])

#     # model = LightGBMModel(lags=len(ts_combined) // 2, output_chunk_length=1, verbose=-1)
#     # model.fit(series=ts_combined_transformed)

#     models[uid] = model

#     # Generate forecasts
#     forecast = model.predict(n=fct_periods, series=ts_combined_transformed)
#     forecasts[uid] = scalers[uid].inverse_transform(forecast)  # Inverse transform to get back to original scale

# # Convert forecasts to df
# lgbm_fct = convert_fct2df(forecasts)

In [17]:
########################
# XTREND - DECAY
########################
def apply_exponential_decay(df, start_date, end_date, end_value_percentage, target_unique_ids):
    # Convert 'ds' to datetime if it's not already
    df['ds'] = pd.to_datetime(df['ds'])
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Function to apply decay for each group
    def decay_group(group):
        # Only apply changes if unique_id is in target_unique_ids
        if group['unique_id'].iloc[0] not in target_unique_ids:
            return group

        # Sort by date to ensure proper indexing
        group = group.sort_values(by='ds')

        # Columns to apply decay to
        decay_columns = [col for col in group.columns if col not in ['unique_id', 'ds']]

        # Initialize a dictionary to keep the end values for each decay column
        end_values = {}

        # Find start and end values and dates for each column
        for col in decay_columns:
            if start_date in group['ds'].values and end_date in group['ds'].values:
                start_value = group.loc[group['ds'] == start_date, col].iloc[0]
                end_value = start_value * end_value_percentage
                end_values[col] = end_value  # Store the end value for this column

                # Calculate the decay rate based on exponential decay formula
                days = (end_date - start_date).days
                decay_rate = np.log(end_value / start_value) / days

                # Apply exponential decay for dates between start_date and end_date
                for date in pd.date_range(start_date, end_date):
                    if date in group['ds'].values:
                        t = (date - start_date).days
                        new_value = start_value * np.exp(decay_rate * t)
                        group.loc[group['ds'] == date, col] = new_value

        # Replace column values for dates after end_date with the respective end values
        for col, end_value in end_values.items():
            if end_value is not None:  # Ensure there was an end value calculated
                group.loc[group['ds'] > end_date, col] = end_value

        return group

    # Apply the decay_group function to each group and return the modified dataframe
    return df.groupby('unique_id').apply(decay_group).reset_index(drop=True)

# Apply exponential decay
# lgbm_fct.rename(columns={'LGBM': 'y'}, inplace=True)
ets_df.rename(columns={'ETS': 'y'}, inplace=True)
arima_df.rename(columns={'ARIMA': 'y'}, inplace=True)

# Micafungin
arima_df = apply_exponential_decay(arima_df, '2023-07-01', '2023-08-01', 0, divested_ids)
ets_df = apply_exponential_decay(ets_df, '2023-07-01', '2023-08-01', 0, divested_ids)

# Lexiscan
arima_df = apply_exponential_decay(arima_df, '2023-04-01', '2023-12-01', .1, loe_ids)
ets_df = apply_exponential_decay(ets_df, '2023-04-01', '2023-12-01', .1, loe_ids)


<ipython-input-17-7d582f2577b1>:51: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  return df.groupby('unique_id').apply(decay_group).reset_index(drop=True)
<ipython-input-17-7d582f2577b1>:51: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  return df.groupby('unique_id').apply(decay_group).reset_ind

In [18]:
########################
# XTREND - GROWTH
########################
volume_act = volume_act[~volume_act['unique_id'].isin(new_ids)]

In [20]:
ets_df.head()

,unique_id,ds,y_0.005,y_0.025,y_0.165,y_0.25,y,y_0.75,y_0.835,y_0.975,y_0.995
0,global/North America/US10 - Astellas Pharma US...,2023-01-01,"17,957,703,363.15","19,437,905,136.97","22,073,298,972.45","24,253,886,730.82","26,989,294,472.57","29,739,847,697.40","30,695,633,947.99","33,082,697,628.70","33,759,749,494.81"
1,global/North America/US10 - Astellas Pharma US...,2023-02-01,"18,974,400,334.86","19,992,838,685.07","22,696,912,998.00","24,170,942,999.30","26,888,171,003.13","29,075,712,143.71","30,155,150,691.10","33,236,955,501.36","34,223,151,740.81"
2,global/North America/US10 - Astellas Pharma US...,2023-03-01,"17,479,505,008.41","18,375,069,832.14","20,561,809,778.00","21,111,472,224.68","25,415,407,334.93","27,999,410,315.45","29,477,365,719.88","33,806,891,264.13","34,247,262,736.64"
3,global/North America/US10 - Astellas Pharma US...,2023-04-01,"17,068,560,245.26","17,522,788,437.90","22,227,839,586.84","22,625,073,339.19","27,395,952,881.37","30,379,135,974.76","31,746,433,204.56","39,892,609,127.00","40,248,469,366.40"
4,global/North America/US10 - Astellas Pharma US...,2023-05-01,"13,660,798,869.23","14,622,353,408.33","17,961,102,380.32","21,358,533,768.66","24,995,350,850.06","27,938,413,905.04","28,868,695,242.62","33,652,644,199.01","33,894,185,842.30"


In [21]:
########################
# METRICS
########################
# Subset
volume_act_xsm = volume_act[['unique_id', 'ds', 'y']]
budget2 = budget[['unique_id', 'ds', 'y']]
ets_df2 = ets_df[['unique_id', 'ds', 'y']]
arima_df2 = arima_df[['unique_id', 'ds', 'y']]

# Assign names
volume_act_xsm.rename(columns={'y': 'Actuals'}, inplace=True)
budget2.rename(columns={'y': 'Budget'}, inplace=True)
# lgbm_fct.rename(columns={'y': 'LGBM'}, inplace=True)
ets_df2.rename(columns={'y': 'ETS'}, inplace=True)
arima_df2.rename(columns={'y': 'ARIMA'}, inplace=True)

# Merge actuals, budget and forecast
rev_at = volume_act_xsm.merge(ets_df2, on=['unique_id', 'ds'], how='left')
rev_at = rev_at.merge(budget2, on=['unique_id', 'ds'], how='left')
# rev_at = rev_at.merge(lgbm_fct, on=['unique_id', 'ds'], how='left')
rev_at = rev_at.merge(arima_df2, on=['unique_id', 'ds'], how='left')

# Conditions for selection
# conditions = [rev_at['unique_id'].isin(spend_ids),rev_at['unique_id'].isin(arima_ids),rev_at['unique_id'].isin(ets_ids)]
# choices = [rev_at['ARIMA'], rev_at['ARIMA'],rev_at['ETS']]

conditions = [rev_at['unique_id'].isin(arima_ids),rev_at['unique_id'].isin(ets_ids)]
choices = [rev_at['ARIMA'],rev_at['ETS']]

# Creating the new column 'SelectedFCT' based on the conditions
rev_at['SelectedFCT'] = np.select(conditions, choices, default=np.nan)

# Only keep tested ts
rev_at = rev_at[rev_at['unique_id'].isin(tested_ts)]

# Filter for dates
data4metrics = rev_at[(rev_at['ds']<=fct_end_date) & (rev_at['ds']>=fct_st_date)]

# Sum up the values for each unique_id
numeric_cols = data4metrics.columns.drop(['unique_id', 'ds'])
summed_df = data4metrics.groupby('unique_id')[numeric_cols].sum()

# Calculate difference and percentage differences from 'Actuals'
absolute_diff = summed_df.subtract(summed_df['Actuals'], axis=0).abs()
percentage_diff = summed_df.subtract(summed_df['Actuals'], axis=0).div(summed_df['Actuals'], axis=0).abs()

# Drop the 'Actuals' column as we don't need to compare it with itself
absolute_diff.drop(columns=['Actuals', 'ARIMA', 'ETS'], inplace=True)

# Find the column with the lowest difference for each unique_id and add to metrics table
min_diff_col = absolute_diff.idxmin(axis=1)
data4metrics['lowest_diff_col'] = data4metrics['unique_id'].map(min_diff_col)

# Find winner
winner = data4metrics.groupby('lowest_diff_col')
# winner = data4metrics[data4metrics['unique_id'].isin(non_spend_ids)].groupby('lowest_diff_col')

# Get Budget winners
bud_winners = winner.get_group('Budget')['unique_id'].unique()
# ets_winners = winner.get_group('ETS')['unique_id'].unique()
# arima_winners = winner.get_group('ARIMA')['unique_id'].unique()
# lgbm_winners = winner.get_group('LGBM')['unique_id'].unique()


winner['unique_id'].nunique()

<ipython-input-21-d696520161a0>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_act_xsm.rename(columns={'y': 'Actuals'}, inplace=True)
<ipython-input-21-d696520161a0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget2.rename(columns={'y': 'Budget'}, inplace=True)
<ipython-input-21-d696520161a0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ets_df2.rename(columns={'y': 'ETS'}, inplace=True)
<ipython-input-21-d6965201

lowest_diff_col
Budget         1
SelectedFCT    1
Name: unique_id, dtype: int64

In [22]:
########################
# CREATE PLOT DATA
########################
fct_st_date = pd.to_datetime(fct_st_date)

# Add revenue actuals
data2plot = rev_at.copy()
data2plot['ds'] = pd.to_datetime(data2plot['ds'])

# Update Actuals columns
data2plot['Actuals (Train)'] = data2plot['Actuals'].copy()
data2plot['Actuals'] = data2plot.apply(lambda row: row['Actuals'] if row['ds'] >= fct_st_date else None, axis=1)
data2plot['Actuals (Train)'] = data2plot.apply(lambda row: row['Actuals (Train)'] if row['ds'] < fct_st_date else None, axis=1)

# Filter to end date
data2plot = data2plot[data2plot['ds']<=fct_end_date]

# Find TS to fix
ts2fix = data2plot[data2plot['unique_id'].isin(bud_winners)]
tsnonspend = data2plot[data2plot['unique_id'].isin(non_spend_ids)]

data2plot.head()

,unique_id,ds,Actuals,ETS,Budget,ARIMA,SelectedFCT,Actuals (Train)
0,global/North America/US10 - Astellas Pharma US...,2014-04-01,NaN,NaN,NaN,NaN,NaN,"5,030,183,962.65"
1,global/North America/US10 - Astellas Pharma US...,2014-05-01,NaN,NaN,NaN,NaN,NaN,"4,848,199,687.80"
2,global/North America/US10 - Astellas Pharma US...,2014-06-01,NaN,NaN,NaN,NaN,NaN,"5,213,402,131.35"
3,global/North America/US10 - Astellas Pharma US...,2014-07-01,NaN,NaN,NaN,NaN,NaN,"6,323,371,010.25"
4,global/North America/US10 - Astellas Pharma US...,2014-08-01,NaN,NaN,NaN,NaN,NaN,"6,160,038,163.65"


In [ ]:
# ########################
# # DECOMPOSITION
# ########################
# import pandas as pd
# from statsmodels.tsa.seasonal import seasonal_decompose, STL

# def decompose_series(df, frequency='M'):
#     # Create an empty DataFrame to store results
#     results_df = pd.DataFrame()

#     # Iterate through each unique_id in the DataFrame
#     for unique_id in df['unique_id'].unique():
#         # Filter the DataFrame for the unique_id
#         group = df[df['unique_id'] == unique_id]

#         # Convert 'ds' to datetime if not already and sort values
#         group = group.sort_values('ds')

#         # Find the first non-zero 'y' value
#         start_index = group.loc[group['y'] > 0].index.min()

#         # Subset the DataFrame from the first non-zero 'y'
#         if pd.notnull(start_index):
#             subset = group.loc[start_index:]

#             # Resample and fill missing values if necessary
#             subset.set_index('ds', inplace=True)
#             subset_resampled = subset.resample(frequency).mean(numeric_only=True).ffill()

#             # Determine the period for decomposition based on the number of observations
#             if len(subset_resampled) < 24:
#                 period = 6  # Ensure period is odd for STL, adjust if necessary
#             else:
#                 period = 12  # Ensure period is odd for STL, adjust if necessary

#             # Make period odd for STL if it's not
#             if period % 2 == 0:
#                 period += 1

#             # Apply traditional decomposition if there are enough data points
#             if len(subset_resampled) >= 2 * period:  # Ensure enough data for traditional decomposition
#                 traditional_decomp = seasonal_decompose(subset_resampled['y'], model='additive', period=max(period, 2))
#                 subset_resampled['traditional_trend'] = traditional_decomp.trend
#                 subset_resampled['traditional_seasonal'] = traditional_decomp.seasonal
#                 subset_resampled['traditional_residual'] = traditional_decomp.resid

#             # Apply STL decomposition if there are enough data points
#             if len(subset_resampled) >= 2 * period:  # Ensure enough data for STL decomposition
#                 stl = STL(subset_resampled['y'], seasonal=period, robust=True)  # Adjust parameters as needed
#                 stl_result = stl.fit()
#                 subset_resampled['stl_trend'] = stl_result.trend
#                 subset_resampled['stl_seasonal'] = stl_result.seasonal
#                 subset_resampled['stl_residual'] = stl_result.resid

#             # Reset index to move 'ds' from index back to column
#             subset_resampled.reset_index(inplace=True)
#             subset_resampled['unique_id'] = unique_id  # Add the unique_id column to the subset

#             # Concatenate the results
#             results_df = pd.concat([results_df, subset_resampled], ignore_index=True)

#     return results_df

# # Apply the function to your DataFrame
# decomposed_results = decompose_series(volume_act, frequency='M')  # You can change the frequency as needed
# decomposed_results.head()


In [23]:
########################
# CREATE HIERARCHICAL DATAFRAMES
########################
def split_unique_id_into_columns(df, column_name):

    # Split 'unique_id' into 4 new columns
    split_columns = df[column_name].str.split('/', expand=True)
    # Rename the columns
    split_columns.columns = ['top_level', 'level2', 'level3', 'bottom_level']
    # Join back to original dataframe
    result_df = pd.concat([df, split_columns], axis=1)

    # Check for columns that are not in split_columns, 'unique_id' or 'ds'
    for col in result_df.columns:
        if col not in ['top_level', 'level2', 'level3', 'bottom_level', 'unique_id', 'ds']:
            # Rename the column to 'y'
            result_df = result_df.rename(columns={col: 'y'})
            break  # Assuming only one column needs to be renamed

    return result_df

def create_hier(data, hierarchy_levels, tested_ts, fct_periods):

    # Filter data based on tested_ts
    data_filtered = data[data['unique_id'].isin(tested_ts)]

    # Identify the columns to aggregate
    columns_to_aggregate = [col for col in data_filtered.columns if col not in ['unique_id', 'ds','top_level', 'level2', 'level3', 'bottom_level']]

    hier_final = data_filtered[['unique_id', 'ds']]
    # Fill NA values for these columns
    for col in columns_to_aggregate:
        data_quantile = data_filtered[['unique_id', 'ds','top_level', 'level2', 'level3', 'bottom_level']+[col]]
        data_quantile.rename(columns={col: 'y'}, inplace=True)
        data_quantile['y'] = data_quantile['y'].fillna(0)

        hier, S_df, tags = aggregate(df=data_quantile, spec=hierarchy_levels)

        hier = hier.reset_index()
        hier.rename(columns={'y': col}, inplace=True)

        hier_final = hier_final.merge(hier, on = ['unique_id', 'ds'], how = 'right')
    # Split the data into train and test sets
    test = hier_final.groupby('unique_id').tail(fct_periods)
    train = hier_final.drop(test.index)

    return train, test, S_df, tags

# Create hierarchies for forecast, actuals and budget
rev_fct = split_unique_id_into_columns(rev_at[rev_at['ds']>=fct_st_date][['unique_id', 'ds', 'SelectedFCT']], 'unique_id')
rev_act = split_unique_id_into_columns(rev_at[['unique_id', 'ds', 'Actuals']], 'unique_id')
rev_bud = split_unique_id_into_columns(rev_at[rev_at['ds']>=fct_st_date][['unique_id', 'ds', 'Budget']], 'unique_id')

revf_train, revf_test, S_df, tags = create_hier(rev_fct, hierarchy_levels, tested_ts, fct_periods)
reva_train, reva_test, S_df, tags = create_hier(rev_act, hierarchy_levels, tested_ts, fct_periods)
bud_train, bud_test, S_df, tags = create_hier(rev_bud, hierarchy_levels, tested_ts, fct_periods)




In [24]:
########################
# HIERARCHICAL RECONCILIATION FOR POINT FORECAST
########################
# Select reconcilers
reconcilers = [
    TopDown(method='forecast_proportions')
    # OptimalCombination(method = 'ols', nonnegative=True)
    # BottomUp()
    # ERM(method='closed')
]

# Rename y to model name
revf_test.rename(columns={'y': 'model'}, inplace=True)

# Reconcile the base predictions
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
revf_rec = hrec.reconcile(Y_hat_df=revf_test.set_index('unique_id'), Y_df=reva_train.set_index('unique_id'),
                          S=S_df, tags=tags)

# Reset Index and columns
revf_rec = revf_rec[['ds', revf_rec.columns[2]]]
revf_rec = revf_rec.reset_index()
revf_rec.columns = ['unique_id', 'ds', 'Forecast_H']

In [25]:
########################
# HIERARCHICAL RECONCILIATION FOR PROBABILISTIC FORECAST
########################
# Get ranges
Y_hier_df, S_df, tags = aggregate(df=volume_act, spec=hierarchy_levels)
Y_hier_df = Y_hier_df.reset_index()

#split train/test sets
Y_test_df  = Y_hier_df.groupby('unique_id').tail(fct_periods)
Y_train_df = Y_hier_df.drop(Y_test_df.index)

# Compute base predictions
fcst = StatsForecast(df=Y_train_df,
                     models=[AutoETS(season_length=12)],
                     freq='MS', n_jobs=-1)

Y_hat_df = fcst.forecast(h=fct_periods, fitted=True, level=quantiles)

Y_fitted_df = fcst.forecast_fitted_values()
Y_fitted_df = Y_fitted_df[['unique_id', 'ds', 'y', 'AutoETS', 'AutoETS-lo-0.995','AutoETS-lo-0.975', 'AutoETS-lo-0.835', 'AutoETS-lo-0.75', 'AutoETS-hi-0.75', 'AutoETS-hi-0.835','AutoETS-hi-0.975', 'AutoETS-hi-0.995']]
Y_fitted_df.columns = ['unique_id', 'ds', 'y', 'model', 'model-lo-99.5','model-lo-97.5', 'model-lo-83.5', 'model-lo-75', 'model-hi-75', 'model-hi-83.5','model-hi-97.5', 'model-hi-99.5']

# Create probabilistic dataframe
arima_df.rename(columns={'ARIMA': 'y'}, inplace=True)
ets_df.rename(columns={'ETS': 'y'}, inplace=True)

arima_dfp = arima_df[arima_df['unique_id'].isin(arima_ids)]
ets_dfp = ets_df[ets_df['unique_id'].isin(ets_ids)]

rev_prb = pd.concat([arima_dfp, ets_dfp])

# Split 'unique_id' into 4 new columns
split_columns = rev_prb['unique_id'].str.split('/', expand=True)

# Rename the columns
split_columns.columns = ['top_level', 'level2', 'level3', 'bottom_level']
rev_prb = pd.concat([rev_prb, split_columns], axis=1)

rev_prb_train, rev_prb_test, S_df, tags = create_hier(rev_prb, hierarchy_levels, tested_ts, fct_periods)


reconcilers = [
    BottomUp(),
# TopDown(method='forecast_proportions')
]

# Rename y to model name
# rev_prb_test.rename(columns={'y': 'model'}, inplace=True)
rev_prb_test.columns = ['unique_id', 'ds', 'model-lo-99.5','model-lo-97.5', 'model-lo-83.5', 'model-lo-75', 'model', 'model-hi-75', 'model-hi-83.5','model-hi-97.5', 'model-hi-99.5']

hrec = HierarchicalReconciliation(reconcilers=reconcilers)
rev_prb_rec = hrec.reconcile(Y_hat_df=rev_prb_test.set_index('unique_id'), Y_df=Y_fitted_df.set_index('unique_id'),
                          S=S_df, tags=tags,level=[75, 83.5, 97.5, 99.5], intervals_method='permbu')

rev_prb_rec.rename(columns={'model': 'Forecast'}, inplace=True)

rev_prb_rec = rev_prb_rec.reset_index()

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
<ipython-input-23-33c9b210f644>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_quantile.rename(columns={col: 'y

In [27]:
########################
# CREATE DATAFRAME TO PLOT
########################
bud_test.rename(columns={'y': 'Budget'}, inplace=True)

# Update Actuals columns
reva = pd.concat([reva_train, reva_test])
reva['Actuals'] = reva.apply(lambda row: row['y'] if row['ds'] >= fct_st_date else None, axis=1)
reva['Actuals (Train)'] = reva.apply(lambda row: row['y'] if row['ds'] < fct_st_date else None, axis=1)

# Update forecast
rev_prb_rec = rev_prb_rec[rev_prb_rec['ds']>=fct_st_date]

# Merge
rev_at_hier = reva.merge(rev_prb_rec[['unique_id', 'ds', 'Forecast']], on=['unique_id', 'ds'], how='left')
rev_at_hier = rev_at_hier.merge(bud_test, on=['unique_id', 'ds'], how='left')
rev_at_hier2plot = rev_at_hier.drop(columns=['y'])

rev_prb_rec = rev_prb_rec[['unique_id', 'ds', 'model-lo-99.5', 'model-lo-97.5', 'model-lo-83.5',
       'model-lo-75', 'Forecast', 'model-hi-75', 'model-hi-83.5',
       'model-hi-97.5', 'model-hi-99.5']]

In [73]:
########################
# METRICS
########################

def sum_of_differences(time_series):
    # Find the index of the first non-zero value
    first_non_zero_index = next((index for index, value in enumerate(time_series) if value != 0), None)

    # Check if there is a non-zero value in the series
    if first_non_zero_index is None:
        return 0  # Return 0 if there are no non-zero values

    # Calculate the sum of the differences after the first non-zero value
    sum_diff = sum(abs(time_series[i] - time_series[i - 1]) for i in range(first_non_zero_index + 1, len(time_series)))

    # Calculate the number of time points after the first non-zero value minus one
    num_points = len(time_series) - first_non_zero_index - 1

    # Avoid division by zero
    if num_points <= 0:
        return 0

    # Return the result
    return np.array((sum_diff / num_points).values())[0][0]


import numpy as np
import pandas as pd
from darts.timeseries import TimeSeries

def metrics(actual_data, forecasted_data, quantiles, weights, sample_columns, stochastic=False):
    # Prepare a list to store WSPL results for each unique_id
    results = []

    # Ensure 'ds' is in datetime format
    actual_data['ds'] = pd.to_datetime(actual_data['ds'])

    for unique_id in actual_data['unique_id'].unique():
        # Filter the actual data
        actual_values = actual_data[(actual_data['unique_id'] == unique_id) & (actual_data['ds'] >= fct_st_date)][['ds', 'y']].tail(fct_periods)

        actual_ts = TimeSeries.from_dataframe(actual_values.set_index('ds'))
        historical_actuals = actual_data[actual_data['unique_id'] == unique_id][['ds', 'y']].drop(actual_values.index)

        historical_ts = TimeSeries.from_dataframe(historical_actuals.set_index('ds'))

        # Filter the forecasted data
        forecasted_values = forecasted_data[forecasted_data['unique_id']==unique_id]
        forecasted_values = forecasted_values.sort_values('ds')

        # Find the unique time points
        unique_times = forecasted_values['ds'].unique()
        num_times = len(unique_times)

        # Define the number of components and samples
        num_components = 1  # 'y'
        num_samples = len(sample_columns)   # Number of forecast columns

        # Initialize the 3D array
        array_3d = np.zeros((num_times, num_components, num_samples))

        # Fill in the array
        for i, time in enumerate(unique_times):
            # Select the corresponding rows from the DataFrame
            row = forecasted_values[forecasted_values['ds'] == time]
            # Extract the sample values and assign them to the array
            samples = row[sample_columns].to_numpy().reshape(num_samples)
            array_3d[i, 0, :] = samples

        # Convert the 'ds' column to datetime
        forecasted_values['ds'] = pd.to_datetime(forecasted_values['ds'])

        # Create a DatetimeIndex from the 'ds' column
        datetime_index = pd.DatetimeIndex(forecasted_values['ds'])

        forecasted_ts = TimeSeries.from_times_and_values(datetime_index, array_3d)

        # Initialize losses dictionary
        losses = {}

        if stochastic:
            # Calculate quantile loss for each quantile if stochastic is True
            for q in quantiles:
                try:
                    losses[q] = quantile_loss(actual_ts, forecasted_ts, q)
                except Exception as e:  # Use appropriate exception handling based on your quantile_loss function
                    print(f"Error calculating quantile loss for {q}: {e}")
                    losses[q] = np.nan
        else:
            # Set all losses to NaN if stochastic is False
            for q in quantiles:
                losses[q] = np.nan

        wspl = sum(weights[q] * losses.get(q, np.nan) for q in quantiles) / sum(weights.values()) if stochastic else np.nan

        # Calculate RMSE and RMSSE
        print()
        mse_metric = mse(actual_ts, forecasted_ts)
        scaled = sum_of_differences(historical_ts)

        rmse_metric = np.sqrt(mse_metric)
        rmsse_metric = np.sqrt((mse_metric / scaled)) if scaled != 0 else np.nan

        # Calculate the overall percentage error
        ope_metric = ope(actual_ts, forecasted_ts)

        # Append the result to the list
        results.append({'unique_id': unique_id, 'WSPL': wspl, 'RMSE': rmse_metric, 'RMSSE': rmsse_metric, 'OPE': ope_metric})

    # Convert the list of results into a DataFrame
    results_df = pd.DataFrame(results)
    return results_df


# Usage:
quantiles = [0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.975, 0.995] # Define your quantiles
weights = {0.005: 1, 0.025: 1, 0.165: 1, 0.250: 1, 0.500: 1, 0.750: 1, 0.835: 1, 0.975: 1, 0.995: 1}
sample_columns_fct = ['model-lo-99.5', 'model-lo-97.5', 'model-lo-83.5', 'model-lo-75','Forecast', 'model-hi-75', 'model-hi-83.5', 'model-hi-97.5','model-hi-99.5']


fct_results = metrics(rev_at_hier[['unique_id', 'ds', 'y']], rev_prb_rec, quantiles, weights, sample_columns_fct, stochastic=True)
bud_results = metrics(rev_at_hier[['unique_id', 'ds', 'y']], bud_test, quantiles, weights, ['Budget'], stochastic=False)

<ipython-input-73-b9eb2523933d>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_data['ds'] = pd.to_datetime(actual_data['ds'])


,unique_id,WSPL,RMSE,RMSSE,OPE
0,global,"2,572,437,727.04","4,134,217,939.26","76,002.94",2.35
1,global/North America,"2,572,437,727.04","4,134,217,939.26","76,002.94",2.35
2,global/North America/US10 - Astellas Pharma US...,"2,572,437,727.04","4,134,217,939.26","76,002.94",2.35
3,global/North America/US10 - Astellas Pharma US...,"2,309,698,135.21","3,723,575,761.39","72,700.73",4.96
4,global/North America/US10 - Astellas Pharma US...,"941,991,105.10","2,615,330,551.21","86,185.25",251.63


In [64]:
# bud_test
bud_test.head()






,unique_id,ds,Budget
0,global,2023-04-01,"30,226,266,193.95"
1,global,2023-05-01,"35,129,608,057.69"
2,global,2023-06-01,"31,730,033,179.97"
3,global,2023-07-01,"30,025,563,528.56"
4,global,2023-08-01,"33,095,277,451.89"


<TimeSeries (DataArray) (ds: 9, component: 1, sample: 9)>
array([[[2.49328064e+10, 2.60746853e+10, 2.83547456e+10, 2.89673724e+10,
         3.18278437e+10, 3.33878523e+10, 3.47283360e+10, 3.73961986e+10,
         3.81525363e+10]],

       [[2.37501073e+10, 2.49926437e+10, 2.76810620e+10, 2.84253599e+10,
         3.21618282e+10, 3.42682669e+10, 3.56290893e+10, 3.68032313e+10,
         3.70139324e+10]],

       [[2.53875929e+10, 2.57380872e+10, 2.70637134e+10, 2.87379727e+10,
         3.01754930e+10, 3.27149308e+10, 3.31373600e+10, 3.71230992e+10,
         3.72463209e+10]],

       [[2.01583975e+10, 2.14954493e+10, 2.62231459e+10, 2.71319869e+10,
         2.83323432e+10, 3.08328925e+10, 3.20015842e+10, 3.45333311e+10,
         3.46040652e+10]],

       [[2.37221837e+10, 2.45960070e+10, 2.76189106e+10, 2.90089095e+10,
         3.02453157e+10, 3.32991173e+10, 3.49218904e+10, 3.82195217e+10,
         3.87350063e+10]],

       [[2.23428249e+10, 2.24393951e+10, 2.41522672e+10, 2.59412464e+10,
         2.73735103e+10, 2.90513019e+10, 2.99480195e+10, 3.29054848e+10,
         3.36621150e+10]],

       [[2.06096238e+10, 2.17510843e+10, 2.51511442e+10, 2.54697234e+10,
         2.76709547e+10, 3.01677726e+10, 3.18666413e+10, 3.47743311e+10,
         3.57823614e+10]],

       [[2.24478021e+10, 2.25260127e+10, 2.48644048e+10, 2.56081002e+10,
         2.71453349e+10, 2.85726129e+10, 2.97325020e+10, 3.20213348e+10,
         3.20805399e+10]],

       [[1.96976677e+10, 1.97522853e+10, 2.31282843e+10, 2.45869309e+10,
         2.70260614e+10, 2.82981230e+10, 2.95545208e+10, 3.40096532e+10,
         3.68639415e+10]]])
Coordinates:
  * ds         (ds) datetime64[ns] 2023-04-01 2023-05-01 ... 2023-12-01
  * component  (component) <U1 '0'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [ ]:
########################
# PLOT
########################
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import pandas as pd

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import base64
from io import BytesIO

# Update the function to include filtering based on 'unique_id'
def plot_data(unique_id):
    # Define x_column and y_columns directly
    x_column = data2use.columns[1]
    y_columns = [data2use.columns[2], data2use.columns[3], data2use.columns[4], data2use.columns[5]]

    # Filter data based on selected unique_id
    filtered_data = data2use[data2use['unique_id'] == unique_id]

    # Set up a 1x3 grid of subplots
    fig, (ax1, ax4) = plt.subplots(1, 2, figsize=(25, 5), gridspec_kw={'width_ratios': [4, 1]}) # Adjust layout for table

    # Plotting multiple y-axes on the first subplot
    for y_column in y_columns:
        ax1.plot(filtered_data[x_column], filtered_data[y_column], label=y_column)
    ax1.set_xlabel(x_column)
    ax1.set_ylabel('Values')
    ax1.set_title(f'Revenue for {unique_id}')
    ax1.legend()

    # Remove axis for table
    ax4.axis('off')
    ax4.axis('tight')

    # Displaying the sum table
    display_data = filtered_data[[x_column] + list(y_columns)].copy()
    display_data = display_data[display_data['ds']>=fct_st_date]
    display_data['ds'] = display_data['ds'].dt.strftime('%m/%d/%Y')

    # Create a sum row
    sum_values = {x_column: 'Sum'}
    for col in list(y_columns):
        sum_values[col] = display_data[col].sum()
    sum_row = pd.DataFrame([sum_values])

    # Create a % diff row
    actuals_sum = sum_values['Actuals']
    pdiff_values = {x_column: '% Diff'}
    for col in list(y_columns):
        pdiff_values[col] = ((display_data[col].sum()-actuals_sum) / actuals_sum) * 100 if actuals_sum != 0 else None
        pdiff_values[col] = round(pdiff_values[col], 2)
    perc_diff_row = pd.DataFrame([pdiff_values])

    # Stack the sum row
    display_data = pd.concat([sum_row, display_data], ignore_index=True)

    # Round the values and add commas
    for column in y_columns:
        if column in display_data.columns:
            # Round to two decimal places
            display_data[column] = display_data[column].round(2)
            # Format with commas
            display_data[column] = display_data[column].apply(lambda x: f"{x:,.2f}")

    # Stack the % diff and remove 'Actuals Train'
    display_data = pd.concat([perc_diff_row, display_data], ignore_index=True)
    display_data = display_data.drop('Actuals (Train)', axis=1)

    # Convert perc_diff_data to array for table
    table_data = display_data.to_numpy()
    # Add table at the right
    table = ax4.table(cellText=table_data, colLabels=display_data.columns, loc='right')
    table.auto_set_font_size(False)
    table.set_fontsize(8.5)  # Set smaller font size if necessary
    table.scale(4, 1.8)  # Adjust scale to fit

    plt.tight_layout()
    plt.show()


# data2use = ts2fix
# data2use = tsnonspend
# data2use = data2plot
data2use = rev_at_hier2plot

# Create widgets
unique_id_selector = widgets.SelectionSlider(
    options=data2use['unique_id'].unique(),
    description='unique_id:',
    orientation='horizontal',
    readout=True
)

# Display interactive plot
interact(plot_data, unique_id=unique_id_selector)